# The thinking behind this code
Here, I'm just looking to combine the JEDI catalog with the CDAW CME catalog. I really only care about the CME speed and mass in CDAW, so I strip my dataframe down to just that. 
By "combine" I really mean that I'd like to see which EVE emission lines (or line combos) are most predictive of CME speed and mass. I'd like to use some machine learning techniques to accomplish this. 
Combining the two catalogs also means matching the CMEs in CDAW to the rows I have in JEDI (or putting in null values where CDAW doesn't have a corresponding CME). 

In [1]:
# Standard modules
import numpy as np
import pandas as pd
from astropy.time import Time
import matplotlib.pyplot as plt
from matplotlib import dates
import seaborn as sns

# Custom modules
from jpm_time_conversions import *
from jpm_logger import JpmLogger
%matplotlib inline
sns.set()
plt.style.use('jpm-dark')

## First things first: I've got to read in the catalogs and do a bit of cleaning
and then take a look at the resultant dataframes

In [2]:
# Read in the JEDI and CDAW catalogs
jedi = pd.read_csv('/Users/jmason86/Dropbox/Research/Postdoc_NASA/Analysis/Coronal Dimming Analysis/JEDI Catalog/jedi_v1.csv', low_memory=False)
cdaw = pd.read_csv('/Users/jmason86/Dropbox/Research/Data/CDAW/Historical CME Data.csv', parse_dates=[['Date', 'Time']])

In [3]:
# Clean the CDAW catalog, strip out the columns I don't care about, convert to numeric data type wherever appropriate
cdaw.index = pd.DatetimeIndex(cdaw['Date_Time'])
cdaw.index.rename('Datetime', inplace=True)
cdaw.drop(['Date_Time', 'Width', 'KE [erg]'], inplace=True, axis=1)
cdaw = cdaw.convert_objects(convert_numeric=True)

/Users/jmason86/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [4]:
# More cleaning: restricting the time range of CDAW to that of JEDI
cdaw = cdaw[jedi['GOES Flare Start Time'][0]: jedi['GOES Flare Start Time'][len(jedi) - 1]]
cdaw.head()

,Position Angle [º],Linear Speed [km/s],Mass [g],Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
Datetime,,,,,,,,,,,,,
2010-05-04 18:30:05,341.0,425.0,5.200000e+13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-05-05 00:30:05,232.0,259.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-05-05 13:31:45,333.0,519.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-05-05 17:06:05,331.0,462.0,1.200000e+14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-05-05 17:54:05,236.0,231.0,7.000000e+13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
jedi.tail()

,Event #,GOES Flare Start Time,GOES Flare Peak Time,GOES Flare Class,Pre-Flare Start Time,Pre-Flare End Time,Flare Interrupt,9.4 Pre-Flare Irradiance [W/m2],13.1 Pre-Flare Irradiance [W/m2],13.3 Pre-Flare Irradiance [W/m2],...,103.2 by 63.0 Fitting Score,103.2 by 71.9 Fitting Score,103.2 by 72.2 Fitting Score,103.2 by 77.0 Fitting Score,103.2 by 79.0 Fitting Score,103.2 by 83.6 Fitting Score,103.2 by 95.0 Fitting Score,103.2 by 97.3 Fitting Score,103.2 by 97.7 Fitting Score,103.2 by 102.6 Fitting Score
4765,5055.0,2014-04-15 17:53:00.000,2014-04-15 17:59:00.000,C7.3,2014-04-13 04:41:00.000,2014-04-13 12:41:00.000,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4766,5056.0,2014-04-15 23:43:00.000,2014-04-15 23:49:00.000,C1.6,2014-04-13 04:41:00.000,2014-04-13 12:41:00.000,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4767,5057.0,2014-04-16 01:10:00.000,2014-04-16 01:18:00.000,C1.9,2014-04-13 04:41:00.000,2014-04-13 12:41:00.000,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4768,5058.0,2014-04-16 01:52:00.000,2014-04-16 01:59:00.000,C3.0,2014-04-13 04:41:00.000,2014-04-13 12:41:00.000,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4769,5059.0,2014-04-16 02:08:00.000,2014-04-16 02:11:00.000,C2.8,2014-04-13 04:41:00.000,2014-04-13 12:41:00.000,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Some postfacto cleaning here... JEDI >v1 will have GOES event latitude and longitude included but v1 doesn't so I need to bolt that on. 

In [6]:
from scipy.io.idl import readsav
goes_flare_events = readsav('/Users/jmason86/Dropbox/Research/Data/GOES/events/GoesEventsC1MinMegsAEra.sav')

In [7]:
jedi.insert(4, 'GOES Latitude', np.nan)
jedi.insert(5, 'GOES Longitude', np.nan)

In [8]:
goes_peak_times_to_add = Time(goes_flare_events['event_peak_time_jd'], format='jd', scale='utc').iso
for i in range(len(jedi)):
    matched_indices = np.where(goes_peak_times_to_add == jedi['GOES Flare Peak Time'][i])
    if len(matched_indices[0]) > 0:
        if goes_flare_events['latitude'][matched_indices[0][0]] != -999:
            jedi['GOES Latitude'][i] = goes_flare_events['latitude'][matched_indices[0][0]][0]
            jedi['GOES Longitude'][i] = goes_flare_events['longitude'][matched_indices[0][0]][0]

/Users/jmason86/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jmason86/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Make a merged catalog (DataFrame)
I am using JEDI as the baseline and will fill in what I can from CDAW. This merged set will of course contain columns in addition to what's in JEDI.

In [9]:
jedicdaw = jedi.copy()
jedicdaw['Has CME'] = False
jedicdaw['Matching CME time to time of'] = np.nan
jedicdaw['CME Time'] = np.nan
jedicdaw['CME Speed [km/s]'] = np.nan
jedicdaw['CME Mass [g]'] = np.nan

## Matching up rows in JEDI and CDAW
To match up the rows in the two catalogs, I am using the standard that Alysha Reinard is, but slightly modified for my case: an event is correlated if the CME CDAW start time is between 2 hours before and 4 hours after the dimming max depth time (mean across emission lines) and within 45º of the flare location (converted to position angle). 

First I'll just define a function to convert flare position in lat/lon to position angle so it can be directly compared with the CME CDAW position

In [10]:
def lat_lon_to_position_angle(longitude, latitude):
    """Function to translate heliocentric coordinates (latitude, longitude) into position angle
    Written by Alysha Reinard. 
    
    Inputs:
        longitude [float]: The east/west coordinate
        latitude [float]: The north/south coordinate
        
    Optional Inputs:
       None

    Outputs:
        position_angle [float]: The converted position angle measured in degrees from solar north, counter clockwise
                                                 
    Optional Outputs:
        None

    Example:
        position_angle = lat_lon_to_position_angle(35, -40)    
    """
    x = longitude * 1.0
    y = latitude * 1.0
    if y != 0:
        pa = np.arctan(-x / y)
    else:
        pa = 3.1415926 / 2.  # limit of arctan(infinity)

    pa = pa * 180.0 / 3.1415926

    if y < 0:
        pa += 180
    if x == 90 and y == 0:
        pa += 180
    if pa < 0:
        pa += 360
        
    if x == 0 and y == 0:
        pa = -1

    return pa

Just to make sure the function isn't buggy, I'll assert some values to sanity check them

In [11]:
assert lat_lon_to_position_angle(0, 0) == -1 # halo
assert lat_lon_to_position_angle(0, -90) == 180 # straight down
assert lat_lon_to_position_angle(0, 90) == 0 # straight up
assert lat_lon_to_position_angle(-90, 0) == 90 # to the left [east]
assert lat_lon_to_position_angle(90, 0) == 270 # to the right [west]

Now I'll compute position angle based on GOES lon, lat and add that as a new JEDI column

In [12]:
jedi.insert(6, 'GOES Converted Position Angle [º]', np.nan)

In [13]:
for i in range(len(jedi)):
    jedi['GOES Converted Position Angle [º]'].iloc[i] = lat_lon_to_position_angle(jedi['GOES Longitude'].iloc[i], jedi['GOES Latitude'].iloc[i]) # value setting on copy warning is ok.. the new computes values do show up in original jedi variable

/Users/jmason86/anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [14]:
jedi.tail()

,Event #,GOES Flare Start Time,GOES Flare Peak Time,GOES Flare Class,GOES Latitude,GOES Longitude,GOES Converted Position Angle [º],Pre-Flare Start Time,Pre-Flare End Time,Flare Interrupt,...,103.2 by 63.0 Fitting Score,103.2 by 71.9 Fitting Score,103.2 by 72.2 Fitting Score,103.2 by 77.0 Fitting Score,103.2 by 79.0 Fitting Score,103.2 by 83.6 Fitting Score,103.2 by 95.0 Fitting Score,103.2 by 97.3 Fitting Score,103.2 by 97.7 Fitting Score,103.2 by 102.6 Fitting Score
4765,5055.0,2014-04-15 17:53:00.000,2014-04-15 17:59:00.000,C7.3,-15.0,-21.0,125.537677,2014-04-13 04:41:00.000,2014-04-13 12:41:00.000,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4766,5056.0,2014-04-15 23:43:00.000,2014-04-15 23:49:00.000,C1.6,-15.0,-17.0,131.423665,2014-04-13 04:41:00.000,2014-04-13 12:41:00.000,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4767,5057.0,2014-04-16 01:10:00.000,2014-04-16 01:18:00.000,C1.9,-14.0,-19.0,126.384351,2014-04-13 04:41:00.000,2014-04-13 12:41:00.000,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4768,5058.0,2014-04-16 01:52:00.000,2014-04-16 01:59:00.000,C3.0,-11.0,-3.0,164.744881,2014-04-13 04:41:00.000,2014-04-13 12:41:00.000,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4769,5059.0,2014-04-16 02:08:00.000,2014-04-16 02:11:00.000,C2.8,NaN,NaN,NaN,2014-04-13 04:41:00.000,2014-04-13 12:41:00.000,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
jedicdaw.insert(6, 'GOES Converted Position Angle [º]', np.nan)
jedicdaw['GOES Converted Position Angle [º]'] = jedi['GOES Converted Position Angle [º]']

I need to match up times: which CMEs occur reasonably close in time to each dimming/flare? First I need to figure out whether to use the dimming or flare time for each event.

In [16]:
dimming_times = jedi.filter(regex='Depth Time')
mean_times = []
for i in range(len(dimming_times)):
    tmp = pd.DatetimeIndex(dimming_times.iloc[i])
    tmp = np.nanmean(pd.DatetimeIndex.to_julian_date(tmp[tmp.notnull()]))
    if not np.isnan(tmp):
        mean_times.append(Time(tmp, format='jd').iso)
        jedicdaw['Matching CME time to time of'].iloc[i] = 'Dimming'
    else:
        mean_times.append(jedi['GOES Flare Peak Time'].iloc[i])
        jedicdaw['Matching CME time to time of'].iloc[i] = 'Flare'

/Users/jmason86/anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/jmason86/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:5: RuntimeWarning: Mean of empty slice


In [17]:
jedi_time = Time(mean_times)
cdaw_time = Time(cdaw.index.values.astype(str))

In [18]:
jedicdaw.head()

,Event #,GOES Flare Start Time,GOES Flare Peak Time,GOES Flare Class,GOES Latitude,GOES Longitude,GOES Converted Position Angle [º],Pre-Flare Start Time,Pre-Flare End Time,Flare Interrupt,...,103.2 by 83.6 Fitting Score,103.2 by 95.0 Fitting Score,103.2 by 97.3 Fitting Score,103.2 by 97.7 Fitting Score,103.2 by 102.6 Fitting Score,Has CME,Matching CME time to time of,CME Time,CME Speed [km/s],CME Mass [g]
0,1.0,2010-05-04 16:15:00.000,2010-05-04 16:29:00.000,C3.6,41.0,23.0,330.708637,2010-05-04 08:29:00.000,2010-05-04 16:29:00.000,True,...,NaN,NaN,NaN,NaN,NaN,False,Dimming,NaN,NaN,NaN
1,2.0,2010-05-05 07:09:00.000,2010-05-05 07:16:00.000,C2.3,NaN,NaN,NaN,2010-05-04 23:16:00.000,2010-05-05 07:16:00.000,True,...,NaN,NaN,NaN,NaN,NaN,False,Dimming,NaN,NaN,NaN
2,3.0,2010-05-05 11:37:00.000,2010-05-05 11:52:00.000,C8.8,NaN,NaN,NaN,2010-05-04 23:16:00.000,2010-05-05 07:16:00.000,True,...,NaN,NaN,NaN,NaN,NaN,False,Dimming,NaN,NaN,NaN
3,4.0,2010-05-05 17:13:00.000,2010-05-05 17:19:00.000,M1.2,42.0,37.0,318.621484,2010-05-04 23:16:00.000,2010-05-05 07:16:00.000,False,...,NaN,NaN,NaN,NaN,NaN,False,Dimming,NaN,NaN,NaN
4,5.0,2010-05-07 07:29:00.000,2010-05-07 07:42:00.000,C2.0,40.0,54.0,306.528854,2010-05-06 23:42:00.000,2010-05-07 07:42:00.000,True,...,NaN,NaN,NaN,NaN,NaN,False,Dimming,NaN,NaN,NaN


In [36]:
for jedi_row_index in range(len(jedi)):
    ind = np.where((cdaw_time.jd <= (4./24. + jedi_time[jedi_row_index].jd)) & (cdaw_time.jd >= (jedi_time[jedi_row_index].jd - 2./24.)))
    if ind[0].size == 1:
        cme_pa = cdaw['Position Angle [º]'].iloc[ind[0]].values[0]
        flare_pa = jedicdaw['GOES Converted Position Angle [º]'].iloc[jedi_row_index]
        if np.abs(cme_pa - flare_pa) < 45:
            jedicdaw['Has CME'].iloc[jedi_row_index] = True
            jedicdaw['CME Time'].iloc[jedi_row_index] = cdaw_time[ind[0]].iso[0]
            jedicdaw['CME Speed [km/s]'].iloc[jedi_row_index] = cdaw['Linear Speed [km/s]'].iloc[ind[0]].values[0]
            jedicdaw['CME Mass [g]'].iloc[jedi_row_index] = cdaw['Mass [g]'].iloc[ind[0]].values[0]
    elif ind[0].size > 1:
        # TODO: Figure out how to decide what to do with multiple matching CMEs -- for now just grabbing the first one
        cme_pa = cdaw['Position Angle [º]'].iloc[ind[0]].values[0]  # Just grabbing first one here
        flare_pa = jedicdaw['GOES Converted Position Angle [º]'].iloc[jedi_row_index]
        if np.abs(cme_pa - flare_pa) < 45:
            jedicdaw['Has CME'].iloc[jedi_row_index] = ind[0].size
            jedicdaw['CME Time'].iloc[jedi_row_index] = cdaw_time[ind[0]].iso[0]
            jedicdaw['CME Speed [km/s]'].iloc[jedi_row_index] = cdaw['Linear Speed [km/s]'].iloc[ind[0]].values[0]  # And just grabbing first one here too
            jedicdaw['CME Mass [g]'].iloc[jedi_row_index] = cdaw['Mass [g]'].iloc[ind[0]].values[0]

/Users/jmason86/anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [47]:
jedicdaw.head()

,Event #,GOES Flare Start Time,GOES Flare Peak Time,GOES Flare Class,GOES Latitude,GOES Longitude,GOES Converted Position Angle [º],Pre-Flare Start Time,Pre-Flare End Time,Flare Interrupt,...,103.2 by 83.6 Fitting Score,103.2 by 95.0 Fitting Score,103.2 by 97.3 Fitting Score,103.2 by 97.7 Fitting Score,103.2 by 102.6 Fitting Score,Has CME,Matching CME time to time of,CME Time,CME Speed [km/s],CME Mass [g]
0,1.0,2010-05-04 16:15:00.000,2010-05-04 16:29:00.000,C3.6,41.0,23.0,330.708637,2010-05-04 08:29:00.000,2010-05-04 16:29:00.000,True,...,NaN,NaN,NaN,NaN,NaN,False,Dimming,NaN,NaN,NaN
1,2.0,2010-05-05 07:09:00.000,2010-05-05 07:16:00.000,C2.3,NaN,NaN,NaN,2010-05-04 23:16:00.000,2010-05-05 07:16:00.000,True,...,NaN,NaN,NaN,NaN,NaN,False,Dimming,NaN,NaN,NaN
2,3.0,2010-05-05 11:37:00.000,2010-05-05 11:52:00.000,C8.8,NaN,NaN,NaN,2010-05-04 23:16:00.000,2010-05-05 07:16:00.000,True,...,NaN,NaN,NaN,NaN,NaN,False,Dimming,NaN,NaN,NaN
3,4.0,2010-05-05 17:13:00.000,2010-05-05 17:19:00.000,M1.2,42.0,37.0,318.621484,2010-05-04 23:16:00.000,2010-05-05 07:16:00.000,False,...,NaN,NaN,NaN,NaN,NaN,False,Dimming,NaN,NaN,NaN
4,5.0,2010-05-07 07:29:00.000,2010-05-07 07:42:00.000,C2.0,40.0,54.0,306.528854,2010-05-06 23:42:00.000,2010-05-07 07:42:00.000,True,...,NaN,NaN,NaN,NaN,NaN,2,Dimming,2010-05-07 08:06:05.000,543.0,5.000000e+14


In [45]:
# Save that work to disk since it takes several minutes
jedicdaw.to_csv('jedicdaw_time_position_match.csv', header=True, index=False, mode='w')

In [ ]:
# Or load it from disk rather than running it again
jedicdaw = pd.read_csv('jedicdaw_time_position_match.csv', low_memory=False)
jedicdaw = jedicdaw.convert_objects(convert_numeric=True)

Figure out how many cases there are with multiple CMEs matched in time and position

In [46]:
multiple_cme_df = jedicdaw.loc[(jedicdaw['Has CME'] > 1), "Has CME"]
single_cme_df = jedicdaw.loc[(jedicdaw['Has CME'] == True), "Has CME"]
no_cme_df = jedicdaw.loc[(jedicdaw['Has CME']) == False, "Has CME"]
print('# multiple CMEs: {0} # single CMEs: {1} # no CMEs {2}'.format(len(multiple_cme_df), len(single_cme_df), len(no_cme_df)))

# multiple CMEs: 416 # single CMEs: 237 # no CMEs 4117


In [39]:
max(multiple_cme_df)

7

5       True
14      True
17      True
30      True
32      True
40      True
56      True
70      True
95      True
97      True
197     True
199     True
250     True
276     True
335     True
336     True
384     True
386     True
387     True
405     True
408     True
409     True
410     True
445     True
463     True
465     True
479     True
483     True
485     True
486     True
        ... 
4115    True
4158    True
4160    True
4166    True
4170    True
4276    True
4277    True
4297    True
4299    True
4329    True
4396    True
4430    True
4435    True
4442    True
4446    True
4447    True
4536    True
4548    True
4580    True
4581    True
4582    True
4583    True
4584    True
4595    True
4651    True
4677    True
4697    True
4752    True
4764    True
4765    True
Name: Has CME, Length: 237, dtype: object